<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Green Manufacturing for vehicles</b>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>Introduction</b></p>

<p style = 'font-size:16px;font-family:Arial'>Since the first automobile, the Real Wheel Motor Company has stood for important automotive innovations. These include, for example, the passenger safety cell with crumple zone, the airbag and intelligent assistance systems. Real Wheel Motor Company applies for nearly 2000 patents per year, making the brand the European leader among premium car makers. These cars are leaders in the premium car industry. With a huge selection of features and options, customers can choose the customized models of their dreams.</p>

<p style = 'font-size:16px;font-family:Arial'>To ensure the safety and reliability of each and every unique car configuration before they hit the road, the engineers have developed a robust testing system. But, optimizing the speed of their testing system for so many possible feature combinations is complex and time-consuming without a powerful algorithmic approach. As one of the world’s biggest manufacturers of premium cars, safety and efficiency are paramount on their production lines.</p>

<p style = 'font-size:16px;font-family:Arial'>The goal is to work with a dataset representing different permutations of Real Wheel Motor Company car features to predict the time it takes to pass testing. This will contribute to speedier testing, resulting in lower carbon dioxide emissions without reducing the standards of the company cars.</p>

<p style = 'font-size:16px;font-family:Arial'>This dataset contains an anonymized set of variables, each representing a custom feature in a car. For example, a variable could be 4WD, added air suspension, or a head-up display. The ground truth is labelled ‘y’ and represents the time (in seconds) that the car took to pass testing for each variable.
</p>


</header>

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>1. Start by connecting to the Vantage system.</b></p>


<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).</p>

In [ ]:
%%capture
# # '%%capture' suppresses the display of installation steps of the following packages
!pip install xgboost==1.7.3
#!pip install colorlover
#!pip install teradataml --upgrade teradataml

<p style = 'font-size:18px;font-family:Arial'><b>**RESTART kernel after installing XGBoost</b></p>
<p style = 'font-size:16px;font-family:Arial'>The above statements may need to be uncommented if you run the notebooks on a platform other than ClearScape Analytics Experience that does not have the libraries installed.  If you uncomment those installs, be sure to restart the kernel after executing those lines. </p>

In [ ]:
import json
import getpass
import pandas as pd

from teradataml.dataframe.dataframe import DataFrame
from teradataml import *

import numpy as np # linear algebra
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
color = sns.color_palette()
import xgboost as xgb

%matplotlib inline


from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from collections import defaultdict
import plotly.offline as offline
import colorlover as cl
offline.init_notebook_mode()

warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, then use down arrow to go to next cell. Begin running steps with Shift + Enter keys.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)
eng.execute('''SET query_band='DEMO=GreenManufacturing.ipynb;' UPDATE FOR SESSION; ''')

<b style = 'font-size:20px;font-family:Arial;color:#E37C4D'>2. Getting Data for This Demo
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage.  You have the option of either running the demo using foreign tables to access the data without using any storage on your environment or downloading the data to local storage which may yield somewhat faster execution, but there could be considerations of available storage.  There are two statements in the following cell, and one is commented out.  You may switch which mode you choose by changing the comment string.</p>    
<p style = 'font-size:16px;font-family:Arial'><b>** Note : Due to the large number of columns the initial table creation and data loading make take more time.</b></p>

In [ ]:
#%run -i ../run_procedure.py "call get_data('DEMO_GreenManufacturing_cloud');"
 # Takes about 50 seconds
%run -i ../run_procedure.py "call get_data('DEMO_GreenManufacturing_local');"
 # Takes about 3 minutes 30 seconds

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>3. Analyze the raw data set</b></p>

<p style = 'font-size:16px;font-family:Arial'>Create a DataFrame to get the data from the table created.</p>
<p style = 'font-size:16px;font-family:Arial'><b><i>** Note : There may be a warning message due to a large number of columns in the dataframe. It's a Warning and not an error. Please ignore the warning</i></b></p>



In [ ]:
datadf = DataFrame(in_schema('DEMO_GreenManufacturing', 'Manufacturing_Data'))
datadf

<p style = 'font-size:16px;font-family:Arial'>The ID column is the ID of the cars, 'y' is the time in seconds which the car took to pass testing for each variable. The variables X0-X8 are categorical variables and the remaining are numerical variables having values of 0 and 1. These are the variables which impact the value of 'y'.</p>


<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>4. Check the impact of Categorical variables on target variable 'y'</b></p>

<p style = 'font-size:16px;font-family:Arial'>Create a DataFrame to get the data from the table created.</p>




In [ ]:
train_df=datadf.to_pandas().reset_index()

In [ ]:
var_name = "X0"
col_order = np.sort(train_df[var_name].unique()).tolist()
plt.figure(figsize=(12,6))
# sns.stripplot(x=var_name, y='y', data=train_df, order=col_order)
sns.countplot(x=var_name, data=train_df, order=col_order)
plt.xlabel(var_name, fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title("Distribution of y variable with "+var_name, fontsize=15)
plt.show()

In [ ]:
var_name = "X1"
col_order = np.sort(train_df[var_name].unique()).tolist()
plt.figure(figsize=(12,6))
sns.stripplot(x=var_name, y='y', data=train_df, order=col_order)
plt.xlabel(var_name, fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title("Distribution of y variable with "+var_name, fontsize=15)
plt.show()

In [ ]:
var_name = "X2"
col_order = np.sort(train_df[var_name].unique()).tolist()
plt.figure(figsize=(12,6))
sns.boxplot(x=var_name, y='y', data=train_df, order=col_order)
plt.xlabel(var_name, fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title("Distribution of y variable with "+var_name, fontsize=15)
plt.show()

In [ ]:
var_name = "X3"
col_order = np.sort(train_df[var_name].unique()).tolist()
plt.figure(figsize=(12,6))
sns.violinplot(x=var_name, y='y', data=train_df, order=col_order)
plt.xlabel(var_name, fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title("Distribution of y variable with "+var_name, fontsize=15)
plt.show()

In [ ]:
var_name = "X4"
col_order = np.sort(train_df[var_name].unique()).tolist()
plt.figure(figsize=(12,6))
sns.violinplot(x=var_name, y='y', data=train_df, order=col_order)
plt.xlabel(var_name, fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title("Distribution of y variable with "+var_name, fontsize=15)
plt.show()

In [ ]:
var_name = "X5"
col_order = np.sort(train_df[var_name].unique()).tolist()
plt.figure(figsize=(12,6))
sns.boxplot(x=var_name, y='y', data=train_df, order=col_order)
plt.xlabel(var_name, fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title("Distribution of y variable with "+var_name, fontsize=15)
plt.show()

In [ ]:
var_name = "X6"
col_order = np.sort(train_df[var_name].unique()).tolist()
plt.figure(figsize=(12,6))
sns.boxplot(x=var_name, y='y', data=train_df, order=col_order)
plt.xlabel(var_name, fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title("Distribution of y variable with "+var_name, fontsize=15)
plt.show()

In [ ]:
var_name = "X8"
col_order = np.sort(train_df[var_name].unique()).tolist()
plt.figure(figsize=(12,6))
sns.barplot(x=var_name, y='y', data=train_df, order=col_order)
plt.xlabel(var_name, fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title("Distribution of y variable with "+var_name, fontsize=15)
plt.show()

In [ ]:
var_name = "ID"
plt.figure(figsize=(12,6))
sns.regplot(x=var_name, y='y', data=train_df, scatter_kws={'alpha':0.5, 's':30})
# sns.barplot(x=var_name, y='y', data=train_df)
plt.xlabel(var_name, fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title("Distribution of y variable with "+var_name, fontsize=15)
plt.show()

<p style = 'font-size:16px;font-family:Arial'>After the initial analysis done on the variables and the value of y based on these variables, let's go ahead and try to predict the value of Y using these variables. Below are some steps that should be done before using any prediction model.</p>

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>5. Check the importance of various features on target variable 'y'</b></p>

<p style = 'font-size:16px;font-family:Arial'>We are using the python xgboost model to check the feature importance.</p>




In [ ]:
warnings.simplefilter(action='ignore', category=UserWarning)
for f in ["X0", "X1", "X2", "X3", "X4", "X5", "X6", "X8"]:
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train_df[f].values)) 
        train_df[f] = lbl.transform(list(train_df[f].values))
        
train_y = train_df['y'].values
train_X = train_df.drop(["ID", "y"], axis=1)


def xgb_r2_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'r2', r2_score(labels, preds)

xgb_params = {
    'eta': 0.05,
    'max_depth': 6,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'silent': 1
}
dtrain = xgb.DMatrix(train_X, train_y, feature_names=train_X.columns.values)
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=100, feval=xgb_r2_score, maximize=True)

# plot the important features #
fig, ax = plt.subplots(figsize=(12,18))
xgb.plot_importance(model, max_num_features=50, height=0.8, ax=ax)
plt.show()

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>6. OrdinalEncoding of the categorical variables</b></p>

<p style = 'font-size:16px;font-family:Arial'>Ordinal encoding is used when the variables in the data are ordinal, ordinal encoding converts each label into integer values and the encoded data represents the sequence of labels.</p>
<p style = 'font-size:16px;font-family:Arial'>Since the variables X0-X8 are categorical, we will need to convert them into numerical to use them in different models. We are using the OrdinalEncoding for this conversion. The <b>TD_OrdinalEncodingFit</b> function identifies distinct categorical values from the input table or a user defined list and returns the distinct categorical values along with the ordinal value for each category.</p>

<p style = 'font-size:16px;font-family:Arial'>The Ordinal encoding will be done for both the Train and Test Datasets.</p>



In [ ]:
# Ordinal encoding for Dataset
query = '''Select * from TD_OrdinalEncodingFit (ON 
DEMO_GreenManufacturing.Manufacturing_Data AS INPUTTABLE
OUT volatile table outputtable(Ordinal_fit_output)
USING TargetColumn('X0','X1','X2','X3','X4','X5','X6','X8')) as dt;
'''
try:
    eng.execute(query)
except:
    eng.execute('DROP TABLE Ordinal_fit_output;')
    eng.execute(query)



<p style = 'font-size:16px;font-family:Arial'>Ordinal encoding transform is used on the ordinal encoding fit data to get the numerical values for the categorical values.</p>
<p style = 'font-size:16px;font-family:Arial'>The <b>TD_OrdinalEncodingTransform</b> function maps the categorical value to a specified ordinal value using the TD_OrdinalEncodingFit output.</p>




In [ ]:
# Ordinal Transform for Dataset
query = '''Create multiset table Ordinal_transform_output as  (SELECT * FROM TD_OrdinalEncodingTransform (
ON DEMO_GreenManufacturing.Manufacturing_Data  AS InputTable
ON Ordinal_fit_output AS FitTable DIMENSION
USING
Accumulate ('id','y')
) as dt) with data primary index("ID");
'''
try:
    eng.execute(query)
except:
    eng.execute('DROP TABLE Ordinal_transform_output;')
    eng.execute(query)

In [ ]:
pd.read_sql('SELECT * FROM Ordinal_transform_output', eng)

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>7. Preparation of Data</b></p>

<p style = 'font-size:16px;font-family:Arial'>In the below steps we are preparing the data by joining the converted categorical features and some other important features to be used in Model Training, Scoring and Evaluation</p>


<p style = 'font-size:16px;font-family:Arial'>We join the converted dataframe and the important numerical features to get the final dataset which will be used for the model.</p>

<p style = 'font-size:16px;font-family:Arial'>Get the output of OrdinalTransform into dataframe.</p>

In [ ]:
OrdTransdf = DataFrame(in_schema('demo_user', 'Ordinal_transform_output'))

<p style = 'font-size:16px;font-family:Arial'>We join the converted dataframe and the original data to get important numerical features.</p>

In [ ]:
datadf=datadf.drop(columns=["X0", "X1","X2","X3","X4","X5","X6","X8"])
datadf_join = OrdTransdf.join(other = datadf, on = ["ID"], how = "left",lsuffix='t1',rsuffix='t2')
datadf_join=datadf_join.drop(columns=["t2_ID", "t2_y"])
datadf_join = datadf_join.assign(ID=datadf_join.t1_ID, y=datadf_join.t1_y)
datadf_join = datadf_join.drop(columns=["t1_ID", "t1_y"])

<p style = 'font-size:16px;font-family:Arial'>Create a final dataframe with only the required important features along with the ID and the response column.</p>

In [ ]:
data_new_df = datadf_join[["ID",
"y",
"X0",
"X1",
"X2",
"X3",
"X4",
"X5",                          
"X6",
"X8",
"X47",
"X314",
"X118",
"X315",
"X127",
"X29",
"X115",
"X351",                           
"X151"]]

In [ ]:
copy_to_sql(df = data_new_df, table_name = 'final_data',if_exists='replace')

<p style = 'font-size:16px;font-family:Arial'>We split the data into Train and Test data.</p>

In [ ]:
query = '''Create table TTS_output as (
SELECT * FROM TD_TrainTestSplit(
ON final_data AS InputTable
USING
IDColumn('Id')
Seed(42)
)AS dt
) with data;
'''
try:
    eng.execute(query)
except:
    eng.execute('DROP TABLE TTS_output;')
    eng.execute(query)

In [ ]:
query = '''Create multiset table final_train_data as (
SELECT * FROM TTS_OUTPUT WHERE TD_IsTrainRow = 1
) with data;
'''
try:
    eng.execute(query)
except:
    eng.execute('DROP TABLE final_train_data;')
    eng.execute(query)

In [ ]:
query = '''Create multiset table final_test_data as (
SELECT * FROM TTS_OUTPUT WHERE TD_IsTrainRow = 0
) with data;
'''
try:
    eng.execute(query)
except:
    eng.execute('DROP TABLE final_test_data;')
    eng.execute(query)

In [ ]:
train_new_df=DataFrame(in_schema('demo_user', 'final_train_data'),index=True,index_label='ID')

In [ ]:
test_new_df=DataFrame(in_schema('demo_user', 'final_test_data'),index=True,index_label='ID')

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>8. Decision Forest </b></p>

<p style = 'font-size:16px;font-family:Arial'>The function is an ensemble algorithm used for classification and regression predictive modelling problems. It is an extension of bootstrap aggregation (bagging) of decision trees. Typically, constructing a decision tree involves evaluating the value for each input feature in the data to select a split point.</p>

<p style = 'font-size:16px;font-family:Arial'>The function reduces the features to a random subset (that can be considered at each split point); the algorithm can force each decision tree in the forest to be very different to improve prediction accuracy. The function uses a training dataset to create a predictive model. The TD_DecisionForestPredict function uses the model created by the TD_DecisionForest function for making predictions. The function supports regression, binary, and multi-class classification.</p>

<p style = 'font-size:16px;font-family:Arial'>Consider the following points:
<li style = 'font-size:16px;font-family:Arial'>All input features are numeric. Convert the categorical columns to numerical columns as preprocessing step.</li>
<li style = 'font-size:16px;font-family:Arial'>For classification, class labels (ResponseColumn values) can only be integers. A maximum of 500 classes is supported for classification.</li>
<li style = 'font-size:16px;font-family:Arial'>Any observation with a missing value in an input column is skipped and not used for training. You can use the TD_SimpleImpute function to assign missing values.</li>

<p style = 'font-size:16px;font-family:Arial'>The number of trees built by TD_DecisionForest depends on the NumTrees, TreeSize, CoverageFactor values, and the data distribution in the cluster. The trees are constructed in parallel by all the AMPs, which have a non-empty partition of data.</p>


In [ ]:
query = '''Create table DF_train as (
SELECT * FROM TD_DecisionForest (
ON final_train_data AS INPUTTABLE partition by ANY
USING
  ResponseColumn('y')
InputColumns('id','X0','X1','X2','X3','X4','X5','X6','X8','X47','X314','X118','X315','X127','X29','X115','X351','X151')
MaxDepth(12)
MinNodeSize(1)
NumTrees(4)
ModelType('REGRESSION')
Seed(1)
Mtry(-1)
MtrySeed(1)
) AS dt
) with data;
'''
try:
    eng.execute(query)
except:
    eng.execute('DROP TABLE DF_train;')
    eng.execute(query)

<p style = 'font-size:16px;font-family:Arial'>TD_DecisionForestPredict function uses the model output by TD_DecisionForest function to analyze the input data and make predictions. This function outputs the probability that each observation is in the predicted class. Processing times are controlled by the number of trees in the model. When the number of trees is more than what can fit in memory, then the trees are cached in a local spool space.</p>


In [ ]:
query = '''
Create table DF_Predict as (
SELECT * FROM TD_DecisionForestPredict (
ON final_test_data AS InputTable PARTITION BY ANY
ON DF_Train AS ModelTable DIMENSION
USING
  IdColumn ('id')
  Detailed('false')
  Accumulate('y')
) AS dt) with data;'''

try:
    eng.execute(query)
except:
    eng.execute('DROP TABLE DF_Predict;')
    eng.execute(query)

In [ ]:
pd.read_sql('SELECT * FROM DF_Predict', eng)

In [ ]:
df_result = DataFrame(in_schema('demo_user', 'DF_Predict'))
df_result_pd=df_result.to_pandas().reset_index().sort_values("ID")
df_result_pd

In [ ]:
plt.figure(figsize=(20,20))
plt.ylabel('Time in Test Cycle', fontsize = 14)
plt.xlabel('Vehicle ID', fontsize = 14)
plt.plot(df_result_pd['ID'][:200], df_result_pd['y'][:200], color='g', label='Actual Value')
plt.plot(df_result_pd['ID'][:200], df_result_pd['prediction'][:200], color='r', label='Predicted Value')
plt.title('Actual vs Predicted using DecisionForest Classification', fontsize = 18)
plt.legend()
# plt.ylim(0,1)
plt.show()

In [ ]:
query = '''
SELECT * FROM TD_RegressionEvaluator(
ON DF_Predict as InputTable
USING
ObservationColumn('y')
PredictionColumn('prediction')
Metrics('RMSE','R2','FSTAT')
DegreesOfFreedom(5,28)
NUMOFINDEPENDENTVARIABLES(15)
) as dt;
'''

pd.read_sql(query, eng)

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>9. XGBoost </b></p>

<p style = 'font-size:16px;font-family:Arial'>The TD_XGBoost function, also known as eXtreme Gradient Boosting, is an implementation of the gradient boosted decision tree designed for speed and performance. It has recently been dominating applied machine learning.</p>

<p style = 'font-size:16px;font-family:Arial'>In gradient boosting, each iteration fits a model to the residuals (errors) of the previous iteration to correct the errors made by existing models. The predicted residual is multiplied by this learning rate and then added to the previous prediction. Models are added sequentially until no further improvements can be made. It is called gradient boosting because it uses a gradient descent algorithm to minimize the loss when adding new models.</p>

<p style = 'font-size:16px;font-family:Arial'>Gradient boosting involves three elements:</p>
<li style = 'font-size:16px;font-family:Arial'>A loss function to be optimized.</li>
<li style = 'font-size:16px;font-family:Arial'>A weak learner to make predictions.</li>
<li style = 'font-size:16px;font-family:Arial'>An additive model to add weak learners to minimize the loss function.</li></p>
<p style = 'font-size:16px;font-family:Arial'>The loss function used depends on the type of problem being solved. For example, regression may use a squared error and binary classification may use binomial. A benefit of the gradient boosting is that a new boosting algorithm does not have to be derived for each loss function. Instead, it provides a generic enough framework that any differentiable loss function can be used. The TD_XGBoost function supports both regression and classification predictive modelling problems. The model that it creates is used in the TD_XGBoostPredict function for making predictions. </p>



In [ ]:
query = '''create table xgb_model as (
SELECT * FROM TD_XGBoost (
ON final_train_data partition by ANY
OUT TABLE MetaInformationTable(xgb_out) 
USING
 ResponseColumn('y')
 InputColumns('id','X0','X1','X2','X3','X4','X5','X6','X8','X47','X314','X118','X315','X127','X29','X115','X351','X151')
 MaxDepth(6)
 MinNodeSize(1)
 NumBoostedTrees(20)
 ModelType('REGRESSION')
 Seed(1) 
 RegularizationLambda(1000)
 ShrinkageFactor(0.5)
 IterNum(10)
 ColumnSampling(0.7) 
) as dt) with data;
'''
try:
    eng.execute(query)
except:
    eng.execute('DROP TABLE xgb_model;')
    eng.execute('DROP TABLE xgb_out;')
    eng.execute(query)

<p style = 'font-size:16px;font-family:Arial'>TD_XGBoostPredict performs prediction for test input data using multiple simple trees in the trained model. The test input data should have the same attributes as used during the training phase, which can be up to 2048. These attributes are used to score based on the trees in the model.</p>
<p style = 'font-size:16px;font-family:Arial'>The output contains prediction for each data point in the test data based on regression or classification. The prediction probability is computed based on the majority vote from participating trees. A higher probability implies a more confident prediction by the model. Majority of the trees result in the same prediction.</p>


In [ ]:
query = '''create table xgb_predict_out as (
SELECT * FROM TD_XGBoostPredict(
ON final_test_data as inputtable partition by ANY
ON xgb_model as modeltable dimension ORDER BY task_index, tree_num, iter, tree_order

USING
 IdColumn('id')
 ModelType('regression')
 accumulate('y')
 ) as dt) with data;
'''
try:
    eng.execute(query)
except:
    eng.execute('DROP TABLE xgb_predict_out;')
    eng.execute(query)

In [ ]:
xgb_result = DataFrame(in_schema('demo_user', 'xgb_predict_out'))
xgb_result

In [ ]:
xgb_result_pd=df_result.to_pandas().reset_index().sort_values("ID")
xgb_result_pd

In [ ]:
plt.figure(figsize=(20,20))
plt.ylabel('Time in Test Cycle', fontsize = 14)
plt.xlabel('Vehicle ID', fontsize = 14)
plt.plot(xgb_result_pd['ID'][:200], train_df['y'][:200], color='g', label='Actual Value')
plt.plot(xgb_result_pd['ID'][:200], xgb_result_pd['prediction'][:200], color='r', label='Predicted Value')
plt.title('Actual vs Predicted using XGBoost Classification', fontsize = 18)
plt.legend()
# plt.ylim(0,1)
plt.show()

<p style = 'font-size:16px;font-family:Arial'>The TD_RegressionEvaluator function computes metrics to evaluate and compare multiple models and summarizes how close predictions are to their expected values.</p>

In [ ]:
query = '''
SELECT * FROM TD_RegressionEvaluator(
ON xgb_predict_out as InputTable
USING
ObservationColumn('confidence_lower')
PredictionColumn('prediction')
Metrics('RMSE','R2','FSTAT')
DegreesOfFreedom(5,48)
NUMOFINDEPENDENTVARIABLES(5)
) as dt;
'''

pd.read_sql(query, eng)

<p style = 'font-size:16px;font-family:Arial'>The Metrics of the regression evaluator has the RMSE, R2 and the F-STAT metrics which are specified in the Metrics.</p>

<p style = 'font-size:16px;font-family:Arial'>Thus here we have used 2 different models to train and predict the data. The Regression evaluator is used to evaluate and compare the models. The Teradata In-Database functions are used for training, prediction and evaluation. In this case since we have sample data the result parameters may not be accurate for these models.</p>  

<p style = 'font-size:16px;font-family:Arial'>Root mean squared error (RMSE)The most common metric for evaluating linear regression model performance is called root mean squared error, or RMSE. The basic idea is to measure how bad/erroneous the model’s predictions are when compared to actual observed values. So a high RMSE is “bad” and a low RMSE is “good”.</p>

<p style = 'font-size:16px;font-family:Arial'>The coefficient of determination — more commonly known as R² — allows us to measure the strength of the relationship between the response and predictor variables in the model. It’s just the square of the correlation coefficient R, so its values are in the range 0.0–1.0. Higher values of R- Squared is Good.</p>

<p style = 'font-size:16px;font-family:Arial'>The metrics specified in the Metrics syntax element are displayed. For FSTAT, the following columns are displayed:</p>
<li style = 'font-size:16px;font-family:Arial'>F_score</li>
<li style = 'font-size:16px;font-family:Arial'>F_Critcialvalue</li>
<li style = 'font-size:16px;font-family:Arial'>p_value</li>
<li style = 'font-size:16px;font-family:Arial'>F_Conclusion.</li></p>

<p style = 'font-size:16px;font-family:Arial'>Here we can see that RMSE is lower for XGBoost than DecisionForest and R-Square is better for Decision Forest as compared to XGBoost.</p> 


<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>10. Cleanup</b></p>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Work Tables</b></p>

In [ ]:
eng.execute('DROP TABLE Ordinal_fit_output;')

In [ ]:
eng.execute('DROP TABLE Ordinal_transform_output;')

In [ ]:
eng.execute('DROP TABLE TTS_output;')

In [ ]:
eng.execute('DROP TABLE final_train_data;')

In [ ]:
eng.execute('DROP TABLE final_test_data;')

In [ ]:
eng.execute('DROP TABLE DF_train;')

In [ ]:
eng.execute('DROP TABLE DF_Predict;')

In [ ]:
eng.execute('DROP TABLE xgb_model;')

In [ ]:
eng.execute('DROP TABLE xgb_out;')

In [ ]:
eng.execute('DROP TABLE xgb_predict_out;')

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Databases and Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_GreenManufacturing');" 
#Takes 40 seconds

<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">Copyright © Teradata Corporation - 2023. All Rights Reserved.</footer>